In [22]:

import pandas as pd

DATA_PATH = "../../Datasets/MAMS-ATSA/Downsampled/test/test_single_row.csv"
TRAIN_PATH = "../../Datasets/MAMS-ATSA/Downsampled/train/train_single_row.csv"
OUTPUT_PATH = "../../Datasets/MAMS-ATSA/Downsampled/test/test_multi_row_few_shot_fix.csv"


In [23]:
def create_terms(df):
    s = ""
    for term in df['term']:
        s += "'"+ term + "'" + "\n"
    return s


In [24]:
def create_output(df):
    # Convert DataFrame to list of lists
    term_sentiments = df[['term', 'polarity']].values.tolist()
    string_format = '{"term_sentiments": [' + ', '.join([f'["{term}", "{sentiment}"]' for term, sentiment in term_sentiments]) + ']}'
    return string_format

In [25]:
df = pd.read_csv(DATA_PATH)
df_train = pd.read_csv(TRAIN_PATH)
df_train_og = pd.read_csv(TRAIN_PATH)

df['example1_term_id'] = None
df['example1_prompt'] = None
df['example2_term_id'] = None
df['example2_prompt'] = None
df['example3_term_id'] = None
df['example3_prompt'] = None


print(df_train.head())
print(df.head())


   text_id  term_id                                               text  \
0        0        0  The decor is not special at all but their food...   
1        0        1  The decor is not special at all but their food...   
2        0        2  The decor is not special at all but their food...   
3        1        3  when tables opened up, the manager sat another...   
4        1        4  when tables opened up, the manager sat another...   

      term  polarity  
0    decor  negative  
1     food  positive  
2   prices  positive  
3   tables   neutral  
4  manager  negative  
   text_id  term_id                                               text  \
0        0        0  Amusing details distinguish desserts, from dul...   
1        0        1  Amusing details distinguish desserts, from dul...   
2        0        2  Amusing details distinguish desserts, from dul...   
3        0        3  Amusing details distinguish desserts, from dul...   
4        1        4  The server was so busy the

In [26]:
examples_sentences = [[None, None], [None, None], [None, None]] 
df_train.drop_duplicates(subset='text', keep='first', inplace=True)
df_train = df_train.reset_index(drop=True)
for i in range(3):
    example_sentence = "Input: \"" + df_train['text'][i] + "\"\nTerms: \n"+ create_terms(df_train_og[df_train_og["text_id"] == df_train['text_id'][i]]) + "\n"+ "Output: " + create_output(df_train_og[df_train_og["text_id"] == df_train['text_id'][i]]) + "\n"
    examples_sentences[i][0] = df_train['text_id'][i]
    examples_sentences[i][1] = example_sentence

In [27]:
for i in examples_sentences:
    print(i[1])

Input: "The decor is not special at all but their food and amazing prices make up for it."
Terms: 
'decor'
'food'
'prices'

Output: {"term_sentiments": [["decor", "negative"], ["food", "positive"], ["prices", "positive"]]}

Input: "when tables opened up, the manager sat another party before us."
Terms: 
'tables'
'manager'

Output: {"term_sentiments": [["tables", "neutral"], ["manager", "negative"]]}

Input: "Though the menu includes some unorthodox offerings (a peanut butter roll, for instance), the classics are pure and great--we've never had better sushi anywhere, including Japan."
Terms: 
'menu'
'peanut butter roll'
'classics'
'sushi'

Output: {"term_sentiments": [["menu", "neutral"], ["peanut butter roll", "negative"], ["classics", "positive"], ["sushi", "positive"]]}



In [28]:
def fill_examples(df, df_train):
    for i in range(len(df)):
        for j in range(len(examples_sentences)):
            df.loc[i, 'example{}_term_id'.format(j+1)] = examples_sentences[j][0]
            df.loc[i, 'example{}_prompt'.format(j+1)] = examples_sentences[j][1]
        print(i)
    return df

df = fill_examples(df, df_train)

print(df.head())


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:

print(df['example1_prompt'][0])
print(df.head())

Input: "The decor is not special at all but their food and amazing prices make up for it."
Terms: 
'desserts'
'dulce de leche ice-cream'
'chocolate sauce tic-tac-toe'
'poached pineapple'

Output: {"term_sentiments": [["desserts", "positive"], ["dulce de leche ice-cream", "neutral"], ["chocolate sauce tic-tac-toe", "neutral"], ["poached pineapple", "neutral"]]}

   text_id  term_id                                               text  \
0        0        0  Amusing details distinguish desserts, from dul...   
1        0        1  Amusing details distinguish desserts, from dul...   
2        0        2  Amusing details distinguish desserts, from dul...   
3        0        3  Amusing details distinguish desserts, from dul...   
4        1        4  The server was so busy the night we visited th...   

                          term  polarity example1_term_id  \
0                     desserts  positive                0   
1     dulce de leche ice-cream   neutral                0   
2  choco

In [ ]:
df.to_csv(OUTPUT_PATH, index = True)